In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

from PIL import Image

PATH = "./data/"
sz=299 #initial training image size
arch = resnet34 #image architecture developed by Facebook AI team #https://github.com/facebookresearch/ResNeXt
bs = 28 #batch size

In [ ]:
!cp -r ../input/dogsandcats/dogscats_small/dogscats_small ./data

In [ ]:
!ls {PATH}

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.backends.cudnn.enabled

In [ ]:
!ls {PATH}

In [ ]:
#Setup image transformation
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

#Transform data
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)

#Load pretrain model
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [ ]:
#training with 0.001 as learning rate and 1 cycle
learn.fit(1e-2, 1)

#trun off precompute
learn.precompute=False

In [ ]:
#cycle_len?
learn.fit(1e-2, 2, cycle_len=1)

In [ ]:
#unfreeze pretrained layers so that can be trained. Explanation?
learn.unfreeze()

#setting different learning rates
lr=np.array([1e-4,1e-3,1e-2])

In [ ]:
#More learning(training), this time allow pretrained layers to be trained
learn.fit(lr, 3, cycle_len=1)

In [ ]:
#Explain TTA time test augmentation

#preicted results are provided as log of the probablity of each, y is the actual label of the images
log_preds,y = learn.TTA()

#taking the averge of predictionas generated by TTA
probs = np.mean(np.exp(log_preds),0)

#calculate accuracy
accuracy_np(probs,y)

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

# def load_img_id(ds, idx, path = PATH): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))
def load_img_id(ds, idx, path = PATH): return np.array(Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask((preds == data.val_y)==is_correct & (data.val_y == y), mult)

In [ ]:
plot_val_with_title(rand_by_correct(True),"Correctly classified")

In [ ]:
ls data/test

In [ ]:
!wget https://images.freeimages.com/images/large-previews/466/cat-1401781.jpg

In [ ]:
test_file = f'{PATH}test/20.jpg'
Image.open(test_file)

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz) #get transformations
im = open_image(test_file) #f'{PATH}test/35.jpg'
im = val_tfms(im)
learn.precompute=False 
preds = learn.predict_array(im[None])
np.argmax(preds)
# 0 for cat, 1 for dog
